# ECON 490: Merge and Append (8)

## Prerequisites 
---
1. Be able to effectively use Stata do files and generate log files.
2. Be able to change your directory so that Stata can find your files.
3. Import datasets in csv and dta format. 
4. Save data files. 
5. Create new variables using the command `egen` and pre-commands`by` and `bysort`.

## Learning objectives:
---
- Adding new variables to an existing dataset (merge).
- Adding new observations to already existing variables (append).

We'll continue working with the fake data dataset introduced in the previous lecture. Recall that this dataset is simulating information of workers in the years 1982-2012 in a fake country where a training program was introduced in 2003 to boost their earnings. 

In [1]:
clear*

use fake_data, clear

## 8.1 Merge 

Before introducing the command `merge`, we need the different steps needed to properly combine datasets.

### 1. Check the dataset's unique identifiers 
The key to merging datasets is to understand what are the variables that *uniquely* identify each observation.

In [3]:
%browse 10

,workerid,year,sex,birth_year,age,start_year,region,treated,earnings
1,1,1999,M,1944,55,1997,1,0,39975.008
2,1,2001,M,1944,57,1997,1,0,278378.06
3,2,2001,M,1947,54,2001,4,0,18682.6
4,2,2002,M,1947,55,2001,4,0,293336.41
5,2,2003,M,1947,56,2001,4,0,111797.26
6,3,2005,M,1951,54,2005,5,0,88351.672
7,3,2010,M,1951,59,2005,5,0,46229.574
8,4,1997,M,1952,45,1997,5,1,24911.029
9,4,2001,M,1952,49,1997,5,1,9908.3623
10,5,2009,M,1954,55,1998,2,1,137207.34


For example, it seems like each observation in the fake_data dataset is identified by the variables workerid and year (worker-year pair). We need to check whether this is true or not using the `duplicates report` command.

In [4]:
duplicates report workerid year


Duplicates in terms of workerid year

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        1 |      2861772             0
--------------------------------------


What this table shows is that there are 2861772 workerid-year combination (which is exactly equal to all of our observations). This means that every observation we have corresponds to a worker in a particular year. 

Let's take a look at a different dataset now.

In [5]:
use region_year_data, clear

In [6]:
%browse 10

,year,region,avg_log_earnings,total_employment
1,1998,1,10.506687,30004
2,1999,1,10.513171,31367
3,2000,1,10.511585,33429
4,2001,1,10.550608,34547
5,2002,1,10.529206,35503
6,2003,1,10.615291,35809
7,2004,1,10.558952,36161
8,2005,1,10.538996,36966
9,2006,1,10.511196,38161
10,2007,1,10.525853,38051


In this case, it seems that every observation corresponds to a region and year combination. Again, we can use `duplicates report` to see if the variables `region` and `year` uniquely identify all observations.

In [7]:
duplicates report region year


Duplicates in terms of region year

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        1 |           70             0
--------------------------------------


Indeed! The table shows that there is not a single case of repeated copies of some observation. Hence, we will refer to these variables as the `unique identifiers`.

### 1. Master and Using Datasets

When merging data we need to decide which dataset will be the dataset (Stata refers to this dataset as `master`) and which will be secondary dataset. The secondary dataset  is were we will get the specific variables we want to duplicate in the `master` dataset and it is refered to as the `using` data.

### 2. Matching observations 
There are three main ways to match observations. The first case is when both observations share the same unique identifiers, so one observation in the master dataset is matched to one observation in the using dataset (reffered as `1:1` merge). The other two cases arise when you match multiple observations in the master dataset to one observation in the using dataset (referred as `m:1` merge). If it is the case that one observation in the master dataset is matched to multiple observations in the using dataset this is known as a `1:m` merge.

### Merging
Once we know the unique identifiers, the master and using datasets and what type of match we are doing we are able to merge the datasets. 

We begin by choosing the master dataset and having it opened in the current Stata session. For the sake of showing an example, let's suppose we want to set fake_data as the `master` dataset, and use region-year  as the `using` dataset. Like we said the fake_data's unique identifiers are workerid and year while the region-year's unique identifiers are region and year. The variables we use to link both datasets have to be the unique identifiers that are present in both datasets. Because `workerid` does not exist in the region-level dataset, we will use variable  `region` and `year` to merge the datasets. 

This would mean that for every region in the using dataset there will be many observations in the individual level (master) dataset to be matched. Therefore, this will be a `m:1` merge.  


In [ ]:
use fake_data, clear

In [10]:
merge m:1 region year using region_year_data


    Result                           # of obs.
    -----------------------------------------
    not matched                       406,963
        from master                   406,963  (_merge==1)
        from using                          0  (_merge==2)

    matched                         2,454,809  (_merge==3)
    -----------------------------------------


Let's analyze the table above. It says that there were 406,963 observations in the master data couldn't be matched to any observation in the using dataset. This is due to the fact that our dataset at the region-year level does not have information for some years. 

Furthermore, the previous table shows that every observation from the using dataset got matched to some observation in the master dataset. The total number of matched observations is roughly 2.5 million. All of this information gets recorded into a new varible named `_merge`. Because of this, it is good practice to write `cap drop _merge` before running a merge command or use the `nogen` option of this command. 

Would we get the same results if we switched the master and using datasets?

In [13]:
use region_year_data, clear
merge 1:m region year using fake_data




    Result                           # of obs.
    -----------------------------------------
    not matched                       406,963
        from master                         0  (_merge==1)
        from using                    406,963  (_merge==2)

    matched                         2,454,809  (_merge==3)
    -----------------------------------------


Indeed, we get the same information. We typically want to restrict to observations that were correctly matched across datasets.

In [14]:
keep if _merge==3

(406,963 observations deleted)


## 8.2 Append

We have used merge to combine datasets horizontally (we have added columns to the `master` dataset). However, if we want to combine datasets vertically  (add observations to the `master` dataset). Adding new information with `mas`  very simple compared to the previous command. When we have a master dataset opened in our session, we can add new informations from another dataset using the `append` command. 

The syntax is simpler than merge 

```
    append using new_dataset
```


This command will add new observations to the variables that are named *the same* across both datasets.

## 8.3 Wrap up

In this module we learned how to combine different datasets. The most important lesson we should take away from this module is that, When we want to merge different variables (columns) from one dataset to another we use the command `merge`. If we want to add observations (rows) from one dataset to another we use the command `append`.

Also, for merging, do not forget to identify the which dataset is the `master` and which is the `using`, which variables are you merging and what are the unique identifiers.